# Muestra Básica de AutoGen

En este ejemplo de código, utilizarás el marco de trabajo de inteligencia artificial [AutoGen](https://aka.ms/ai-agents/autogen) para crear un agente básico.

El objetivo de este ejemplo es mostrarte los pasos que luego utilizaremos en los ejemplos de código adicionales al implementar los diferentes patrones de agentes.


## Importar los paquetes de Python necesarios


In [1]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Crear el Cliente

En este ejemplo, usaremos [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acceder al LLM.

El `model` está definido como `gpt-4o-mini`. Intenta cambiar el modelo a otro disponible en el mercado de GitHub Models para ver diferentes resultados.

Como una prueba rápida, simplemente ejecutaremos un prompt sencillo: `¿Cuál es la capital de Francia?`.


In [2]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

c:\Github\Local\ai-agents-for-beginners_cdc\.venv\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


## Definiendo el Agente

Ahora que hemos configurado el `client` y confirmado que está funcionando, vamos a crear un `AssistantAgent`. A cada agente se le puede asignar:  
**name** - Un nombre corto que será útil para referenciarlo en flujos con múltiples agentes.  
**model_client** - El cliente que creaste en el paso anterior.  
**tools** - Herramientas disponibles que el Agente puede usar para completar una tarea.  
**system_message** - El metaprompt que define la tarea, el comportamiento y el tono del LLM.  

Puedes cambiar el mensaje del sistema para ver cómo responde el LLM. Cubriremos `tools` en la Lección #4.


In [3]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Ejecutar el Agente

La función a continuación ejecutará el agente. Usamos el método `on_message` para actualizar el estado del Agente con el nuevo mensaje.

En este caso, actualizamos el estado con un nuevo mensaje del usuario que es `"Planéame unas excelentes vacaciones soleadas"`.

Puedes cambiar el contenido del mensaje para ver cómo responde de manera diferente el LLM.


In [4]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
